In [195]:
import pandas as pd
import gurobipy as gp
import math
import pickle as pkl
from datetime import datetime, timedelta

In [196]:
product_type = "frozen"
num_time_periods = 7

In [197]:
article_data = pd.read_csv("./data/article.csv")
article_data = article_data[(article_data['TEMPERATURE_ZONE'] == product_type)]
article_data = article_data.head(5)
print(article_data)
# article_data['MEAN_SHELF_LIFE'] = (article_data['MEAN_SHELF_LIFE'] - article_data['MEAN_SHELF_LIFE'].min()) / (article_data['MEAN_SHELF_LIFE'].max() - article_data['MEAN_SHELF_LIFE'].min())
article_data

   ARTICLE_ID TEMPERATURE_ZONE CATEGORY_LEVEL_1         CATEGORY_LEVEL_2   
9    468a73f3           frozen         Tiefkühl        Kartoffelprodukte  \
14   3bd76e22           frozen         Tiefkühl        Pizza & Baguettes   
23   84293966           frozen         Tiefkühl  Fertiggerichte & Snacks   
60   1.35E+70           frozen         Tiefkühl  Fleisch, Fisch & Veggie   
85   d8d111cd           frozen         Tiefkühl                      Eis   

    VOLUME_M3_PER_CU  MEAN_SHELF_LIFE  CU_PER_TU  ORDERING_COST_FIXED   
9           0.001932             1000          1                  3.7  \
14          0.001825             1000         22                 18.9   
23          0.001452             1000         18                 16.2   
60          0.001576             1000          1                  4.1   
85          0.001016             1000         17                 15.1   

    ORDERING_COST_PER_TU  SALES_MARGIN_PER_CU  CLEARING_COST_PER_CU   
9                    0.1         

,ARTICLE_ID,TEMPERATURE_ZONE,CATEGORY_LEVEL_1,CATEGORY_LEVEL_2,VOLUME_M3_PER_CU,MEAN_SHELF_LIFE,CU_PER_TU,ORDERING_COST_FIXED,ORDERING_COST_PER_TU,SALES_MARGIN_PER_CU,CLEARING_COST_PER_CU,MINIMUM_ORDER_QUANTITY_TU,MAXIMUM_ORDER_QUANTITY_TU
9,468a73f3,frozen,Tiefkühl,Kartoffelprodukte,0.001932,1000,1,3.7,0.1,0.01,0.1,0,NaN
14,3bd76e22,frozen,Tiefkühl,Pizza & Baguettes,0.001825,1000,22,18.9,2.9,0.46,1.9,0,NaN
23,84293966,frozen,Tiefkühl,Fertiggerichte & Snacks,0.001452,1000,18,16.2,2.4,0.38,1.5,0,NaN
60,1.35E+70,frozen,Tiefkühl,"Fleisch, Fisch & Veggie",0.001576,1000,1,4.1,0.2,0.03,0.1,0,NaN
85,d8d111cd,frozen,Tiefkühl,Eis,0.001016,1000,17,15.1,2.2,0.35,1.4,0,NaN


In [198]:
# constants definitions
if(product_type=="frozen"):
    warehouse_volume = 50
if(product_type=="chilled"):
    warehouse_volume = 300
if(product_type=="ambient"):
    warehouse_volume = 900
buffer_cost = 25
default_max_order = 10000

In [199]:
def createParameterMatrix(data, columns):
    parameters = []
    for column in columns:
        parameters.append(data[column].to_list())
    parameters = list(map(list, zip(*parameters)))
    return parameters

In [200]:
articles = article_data['ARTICLE_ID'].to_list()

parameters = createParameterMatrix(
    article_data,
    [
        'TEMPERATURE_ZONE',
        'VOLUME_M3_PER_CU',
        'MEAN_SHELF_LIFE',
        'CU_PER_TU',
        'ORDERING_COST_FIXED',
        'ORDERING_COST_PER_TU',
        'CLEARING_COST_PER_CU',
        'MINIMUM_ORDER_QUANTITY_TU',
        'MAXIMUM_ORDER_QUANTITY_TU'
    ]
)
parameters_dict = dict(zip(articles, parameters))

In [201]:
items, category, volume_per_cu, shelf_life, cu_per_tu, ordering_cost_fixed, ordering_cost_per_tu, clearing_cost_per_cu, minimum_order_quantity_tu, maximum_order_quantity_tu = gp.multidict(parameters_dict)

In [202]:
forecast_data = pd.read_csv('./data/sales_'+str(num_time_periods)+'.csv')
forecast_data = forecast_data[forecast_data['ARTICLE_ID'].isin(articles)]

In [203]:
# Create a new dataframe with all dates
all_dates_df = pd.DataFrame({'DATE': pd.date_range(start='2022-06-13', end='2022-06-18')})

# Group the original dataframe by item
grouped = forecast_data.groupby('ARTICLE_ID')

# Initialize an empty list to store the new dataframes
new_dfs = []

# Loop over each group
for item, group_df in grouped:
    
    group_df['DATE'] = pd.to_datetime(group_df['DATE'])

    # Merge the group dataframe with the all_dates dataframe
    merged_df = pd.merge(all_dates_df, group_df, on='DATE', how='outer')
    merged_df['ARTICLE_ID'] = item
    
    # Fill in missing values
    merged_df['PICKING_QUANTITY_CU'] = merged_df['PICKING_QUANTITY_CU'].fillna(0)
    
    # Sort by date and append to the list
    new_dfs.append(merged_df.sort_values('DATE'))
    
# Concatenate all new dataframes into a single dataframe
forecast_data = pd.concat(new_dfs)
time_periods = forecast_data['DATE'].unique()
demand = forecast_data.groupby('ARTICLE_ID').apply(lambda x: dict(zip(x['DATE'], x['PICKING_QUANTITY_CU']))).to_dict()

In [204]:
shelf_life['468a73f3'] = 3
cu_per_tu['468a73f3'] = 2
for key in demand['468a73f3'].keys():
    demand['468a73f3'][key] = 0
demand['468a73f3'][time_periods[0]] = 5
demand['468a73f3']

{Timestamp('2022-06-13 00:00:00'): 5,
 Timestamp('2022-06-14 00:00:00'): 0,
 Timestamp('2022-06-15 00:00:00'): 0,
 Timestamp('2022-06-16 00:00:00'): 0,
 Timestamp('2022-06-17 00:00:00'): 0,
 Timestamp('2022-06-18 00:00:00'): 0}

In [205]:
volume_per_cu['468a73f3'] = 100
warehouse_volume = 0

In [206]:
# model object
m = gp.Model()

# decision variables
# Xit
orders = m.addVars(items, time_periods, vtype=gp.GRB.INTEGER, lb=0)
# Yit
ordered_boolean = m.addVars(items, time_periods, vtype=gp.GRB.BINARY, lb=0)
# Sit
storage_used = m.addVars(items, time_periods, vtype=gp.GRB.INTEGER, lb=0)
# Zt
buffer_storage_used = m.addVars(time_periods, vtype=gp.GRB.INTEGER, lb=0)
# Dit
clearances = m.addVars(items, time_periods, vtype=gp.GRB.INTEGER, lb=0)
# Xit1t2
orders_disaggregated = m.addVars(items, time_periods, time_periods, vtype=gp.GRB.INTEGER, lb=0)
# Sit1t2
storage_disaggregated = m.addVars(items, time_periods, time_periods, vtype=gp.GRB.INTEGER, lb=0)
# Dit1t2
clearance_disaggregated = m.addVars(items, time_periods, time_periods, vtype=gp.GRB.INTEGER, lb=0)

# objective function
ordering_cost_per_tu_objective = gp.quicksum(ordering_cost_per_tu[item] * orders[item, time_period] for item in items for time_period in time_periods)
ordering_cost_fixed_objective = gp.quicksum(ordering_cost_fixed[item] * ordered_boolean[item, time_period] for item in items for time_period in time_periods)
buffer_storage_objective = gp.quicksum(buffer_cost * buffer_storage_used[time_period] for time_period in time_periods)
clearance_objective = gp.quicksum(clearing_cost_per_cu[item] * clearances[item, time_period] for item in items for time_period in time_periods)

m.setObjective(ordering_cost_per_tu_objective + ordering_cost_fixed_objective + buffer_storage_objective, sense=gp.GRB.MINIMIZE)

# new constraints
# demand satisfaction
for item in demand.keys():
    for time_period_iterator, time_period in enumerate(time_periods):
        end_time_period_iterator = min(len(time_periods),int(time_period_iterator+shelf_life[item]))
        m.addConstr(
            storage_used[item, time_periods[time_period_iterator-1]] 
            + 
            gp.quicksum(orders_disaggregated[item, time_period, t2] for t2 in time_periods[time_period_iterator:end_time_period_iterator])
            ==
            storage_used[item, time_period]
            +
            demand[item][time_period]
            +
            clearances[item, time_period]
        )

for item in items:
    for time_period_iterator, time_period in enumerate(time_periods):
        end_time_period_iterator = min(len(time_periods),int(time_period_iterator+shelf_life[item]))
        m.addConstr(
            storage_used[item, time_period]
            ==
            gp.quicksum(orders_disaggregated[item, t1, t2] for t1 in time_periods[:time_period_iterator] for t2 in time_periods[time_period_iterator+1:end_time_period_iterator])
            +
            gp.quicksum(clearance_disaggregated[item, t1, t2] for t1 in time_periods[:time_period_iterator] for t2 in time_periods[time_period_iterator+1:])
        )

# orders disaggregated linking
for item in items:
    for time_period_iterator, time_period in enumerate(time_periods):
        end_time_period_iterator = min(len(time_periods),int(time_period_iterator+shelf_life[item]))
        m.addConstr(
            orders[item, time_period] * cu_per_tu[item]
            ==
            gp.quicksum(orders_disaggregated[item, time_period, t2] for t2 in time_periods[time_period_iterator:end_time_period_iterator])
            +
            gp.quicksum(clearance_disaggregated[item, time_period, t2] for t2 in time_periods[time_period_iterator:])
        )

# clearance disaggregated linking
for item in items:
    for time_period_iterator, time_period in enumerate(time_periods):
        if(time_period_iterator != len(time_periods)-1):
            m.addConstr(
                clearances[item, time_period] 
                ==
                gp.quicksum(clearance_disaggregated[item, t1, time_period] for t1 in time_periods[:time_period_iterator+1])
            )   

# # constraints
# # demand satisfaction
# for item in demand.keys():
#     for time_period_iterator, time_period in enumerate(time_periods):
#         if(time_period_iterator==0):
#             m.addConstr(orders[item, time_period] * cu_per_tu[item] - clearances[item, time_period]>= demand[item][time_period],name="demand constraint_" + str(time_period))
#         else:
#             m.addConstr((orders[item, time_period] * cu_per_tu[item]) + storage_used[item, time_periods[time_period_iterator-1]] - clearances[item, time_period]>= demand[item][time_period],name="demand constraint_" + str(time_period))

# inventory volume constraint
for time_period_iterator, time_period in enumerate(time_periods):
    if(time_period_iterator==0):
        m.addConstr(gp.quicksum(volume_per_cu[item] * ((cu_per_tu[item] * (orders[item, time_period]))) for item in items) <= warehouse_volume + buffer_storage_used[time_period])
    else:
        m.addConstr(gp.quicksum(volume_per_cu[item] * (storage_used[item, time_periods[time_period_iterator-1]] + (cu_per_tu[item] * (orders[item, time_period]))) for item in items) <= warehouse_volume + buffer_storage_used[time_period])

# # inventory disaggregated linking
# for item in items:
#     for time_period in time_periods:
#         m.addConstr(storage_used[item, time_period] == gp.quicksum(storage_disaggregated[item, t1, time_period] for t1 in time))

# # # min/max constraints (linking too)
for item in demand.keys():
    for time_period in time_periods:
        m.addConstr(orders[item, time_period] >= minimum_order_quantity_tu[item] * ordered_boolean[item, time_period])
        # if((not math.isnan(maximum_order_quantity_tu[item]))):
        #     m.addConstr(orders[item, time_period] <= maximum_order_quantity_tu[item] * ordered_boolean[item, time_period])
        # else:
        #     m.addConstr(orders[item, time_period] <= default_max_order * ordered_boolean[item, time_period])

# # current inventory constraint
# for item in demand.keys():
#     for time_period_iterator,time_period in enumerate(time_periods):
#         if(time_period_iterator==0):
#             m.addConstr(storage_used[item, time_periods[time_period_iterator]] == (orders[item, time_period] * cu_per_tu[item]) - demand[item][time_period])
#         else:
#             m.addConstr(storage_used[item, time_periods[time_period_iterator]] == storage_used[item, time_periods[time_period_iterator-1]] + (orders[item, time_period] * cu_per_tu[item]) - demand[item][time_period])

# # clearance constraint
# for item in demand.keys():
#     life = shelf_life[item]
#     for time_period in time_periods[:-life]:
#         m.addConstr(clearances[item, time_period] <= (orders[item, time_period] * cu_per_tu[item]) - (gp.quicksum(orders_disaggregated[item, time_period, t2] for t2 in time_periods[time_period_iterator:time_period_iterator+life])))

m.setParam(gp.GRB.Param.Heuristics, 0) 
m.optimize()

Set parameter Heuristics to value 0
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: AMD Ryzen 7 5800U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 139 rows, 666 columns and 795 nonzeros
Model fingerprint: 0x6f983817
Variable types: 0 continuous, 666 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e-03, 2e+02]
  Objective range  [1e-01, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 5e+01]
Presolve removed 74 rows and 492 columns
Presolve time: 0.00s
Presolved: 65 rows, 174 columns, 489 nonzeros
Variable types: 0 continuous, 174 integer (3 binary)

Root relaxation: objective 1.257752e+04, 63 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 12577.5237    0   22     

In [207]:
print(orders[('468a73f3',time_periods[0])])
print(demand['468a73f3'][time_periods[0]])
print(orders_disaggregated['468a73f3',time_periods[0],time_periods[0]])
print(clearances['468a73f3',time_periods[3]])
for time_period in time_periods:
    if (clearances['468a73f3',time_period].X > 0):
        print("YES")
    for t1 in time_periods:
        if(clearance_disaggregated['468a73f3', t1, time_period].X > 0):
            print(t1)
            print(time_period)
            print("YO")

<gurobi.Var C0 (value 3.0)>
5
<gurobi.Var C126 (value 5.0)>
<gurobi.Var C99 (value 0.0)>
YES
2022-06-13 00:00:00
2022-06-14 00:00:00
YO
YES


In [208]:
buffer_storage_used

{Timestamp('2022-06-13 00:00:00'): <gurobi.Var C90 (value 601.0)>,
 Timestamp('2022-06-14 00:00:00'): <gurobi.Var C91 (value 201.0)>,
 Timestamp('2022-06-15 00:00:00'): <gurobi.Var C92 (value 101.0)>,
 Timestamp('2022-06-16 00:00:00'): <gurobi.Var C93 (value 100.0)>,
 Timestamp('2022-06-17 00:00:00'): <gurobi.Var C94 (value 101.0)>,
 Timestamp('2022-06-18 00:00:00'): <gurobi.Var C95 (value 101.0)>}